<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/rest_api_topics_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok


In [2]:
!git clone https://github.com/NikolasGialitsis/LDA2vec

Cloning into 'LDA2vec'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 187 (delta 40), reused 0 (delta 0), pack-reused 112
Receiving objects: 100% (187/187), 15.71 MiB | 16.13 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [3]:
# -*- coding: utf-8 -*-
"""LDA2vec_inference.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1szh4AuxOleuFERJPzlqehK-yPRVwka2P
"""


!pip install spacy
!pip install jellyfish
!pip install -r /content/requirements.txt
!pip install pylda2vec




     |████████████████████████████████| 143kB 5.8MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp36-cp36m-linux_x86_64.whl size=72997 sha256=b314f6284d7524cbf4726f4b3c11ec47b84a61651e35acf48a7146359ceb046d
  Stored in directory: /root/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
Successfully built jellyfish
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/requirements.txt'


In [4]:
 # Commented out IPython magic to ensure Python compatibility.
%cd /content/LDA2vec/
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import json

from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

import h5py
%cd LDA2vec/
!python -m spacy download en

/content/LDA2vec
Using GPU:0
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[Errno 2] No such file or directory: 'LDA2vec/'
/content/LDA2vec
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [153]:
def getTopicComposition(query):

  trained = h5py.File('/content/LDA2vec/lda2vec.hdf5', 'r+')
  mixture = trained['mixture']
  for key in mixture.keys():
    print('\t',key)
  factors = mixture.get('factors')['W'][()]
  #weights = mixture.get('weights')['W'][()]
  #sampler = trained['sampler'].get('W')[()]
  
 
  sampler = pickle.load(open('/content/LDA2vec/sampler.pkl', 'rb'))
  #factors = pickle.load(open('/content/LDA2vec/factors.pkl', 'rb'))
  vectors = np.load('/content/LDA2vec/vectors.npy')
  old_flattened = np.load('/content/LDA2vec/flattened.npy')
  vocab =  pickle.load(open('/content/LDA2vec/vocab.pkl', 'rb'))
  corpus = pickle.load(open('/content/LDA2vec/corpus.pkl', 'rb'))
  n_vocab = old_flattened.max() + 1

  titles = []
  abstracts = []
  concats = []
  concat = query.split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  print('title:',title)
  print('abstract',abstract[:100])
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)

  stop_words = set(stopwords.words('english')) 
  new_text = []
  for text in concats.copy():
    word_tokens = word_tokenize(text) 
    #remove stopwords
    text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
    #remove punctuation
    #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

    text = ''.join([w for w in text if w not in string.punctuation])
    new_text.append(text) 

  max_length = 10000
  
  '''
  corpus = Corpus()
  
  tokens, vocab = preprocess.tokenize(new_text, max_length, merge=False,n_threads=4)
  
  # Make a ranked list of rare vs frequent words
  corpus.update_word_count(tokens)
  corpus.finalize()
  # The tokenization uses spaCy indices, and so may have gaps
  # between indices for words that aren't present in our dataset.
  # This builds a new compact index
  compact = corpus.to_compact(tokens)
  # Remove extremely rare words
  pruned = corpus.filter_count(compact, min_count=0)
  # Convert the compactified arrays into bag of words arrays
  bow = corpus.compact_to_bow(pruned)
  # Words tend to have power law frequency, so selectively
  # downsample the most prevalent words
  clean = corpus.subsample_frequent(pruned)
  # Now flatten a 2D array of document per row and word position
  # per column to a 1D array of words. This will also remove skips
  # and OoV words
  doc_ids = np.arange(pruned.shape[0])
  flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

  #vectors, s, f = corpus.compact_word_vectors(vocab, model = word2vec_model)
  # vectors = np.delete(vectors,77743,0)
  # Model Parameters
  # Number of documents
  n_docs = doc_ids.max() + 1
  # Number of unique words in the vocabulary
  n_vocab=flattened.max()  + 1

  doc_idx, lengths = np.unique(doc_ids, return_counts=True)
  doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
  doc_lengths[doc_idx] = lengths
  # Count all token frequencies
  tok_idx, freq = np.unique(flattened, return_counts=True)
  term_frequency = np.zeros(n_vocab, dtype='int32')
  term_frequency[tok_idx] = freq


  # Get the string representation for every compact key
  words = corpus.word_list(vocab)[:n_vocab]
  '''
  

  new_text = new_text[0].split()
  tokens = np.zeros(max_length)
  print(n_vocab)
  for ident in range(len(new_text)):
    word = new_text[ident]
    if word in vocab.values():
      for k in vocab.keys():
        if word == vocab[k] and k <= n_vocab:
          tokens[ident] = k

  words = corpus.word_list(vocab)[:n_vocab]
  doc_ids = np.zeros(len(tokens))
  doc_ids = np.array([int(d) for d in doc_ids])

  flattened = tokens.copy()
  n_docs = doc_ids.max() + 1
  
  clambda = 200.0
  # Number of topics to fit
  n_topics = int(os.getenv('n_topics', 10))
  batchsize = 16
  # Power for neg sampling
  power = float(os.getenv('power', 0.75))
  # Intialize with pretrained word vectors
  pretrained = bool(int(os.getenv('pretrained', True)))
  # Sampling temperature
  temperature = float(os.getenv('temperature', 1.0))
  # Number of dimensions in a single word vector
  n_units = int(os.getenv('n_units', 30))
  
  
  # Get the string representation for every compact key
  # How many tokens are in each document
  doc_idx, lengths = np.unique(doc_ids, return_counts=True)
  doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
  doc_lengths[doc_idx] = lengths

  flattened = np.array([int(f) for f in flattened])
  tok_idx, freq = np.unique(flattened, return_counts=True)
  term_frequency = np.zeros(n_vocab, dtype='int32')
  term_frequency[tok_idx] = freq
  

  model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                  n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                  n_samples=15, power=power, temperature=temperature)
  model.sampler.W.data[:, :] = vectors[:n_vocab, :]
  #model.mixture.factors.W.data = factors
  npz = np.load(open('/content/LDA2vec/topics.pyldavis.npz', 'rb'))
  dat = {k: v for (k, v) in npz.items()}

  model.to_gpu()
  optimizer = O.Adam()
  optimizer.setup(model)
  clip = chainer.optimizer.GradientClipping(5.0)
  optimizer.add_hook(clip)


  j = 0
  epoch = 0
  fraction = batchsize * 1.0 / flattened.shape[0]
  progress = shelve.open('progress.shelve')

  
  
  n_epochs = 5
  for epoch in range(n_epochs):
      data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu( model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
      data['doc_lengths'] = doc_lengths
      data['term_frequency'] = term_frequency
      print(data['topic_term_dists'].shape,dat['topic_term_dists'].shape)
      data['topic_term_dists'] = dat['topic_term_dists']
      print('Epoch = ',epoch)
      for d, f in utils.chunks(batchsize, doc_ids, flattened):
          t0 = time.time()
          model.cleargrads()
          #optimizer.use_cleargrads(use=True)
          l = model.fit_partial(d.copy(), f.copy(),update_only_docs=True)
          #print("after partial fitting:", l)
          prior = model.prior()
          loss = prior * fraction
          loss.backward()
          optimizer.update()
          msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} ""P:{prior:1.3e} R:{rate:1.3e}")
          prior.to_cpu()
          loss.to_cpu()
          t1 = time.time()
          dt = t1 - t0
          rate = batchsize / dt
          logs = dict(loss=float(l), epoch=epoch, j=j,
                      prior=float(prior.data), rate=rate)
          #print(msg.format(**logs))
          j += 1

  
  top_n = 10
  topic_to_topwords = {}
  print(data['topic_term_dists'].shape)
  for j, topic_to_word in enumerate(data['topic_term_dists']):
      top = np.argsort(topic_to_word)[::-1][:top_n]
      msg = 'Topic %i '  % j
      top_words = [data['vocab'][i].strip()[:35] for i in top]
      msg += ' '.join(top_words)
      print(msg)
      topic_to_topwords[j] = top_words

  ret_val = ""
  t = 0
  for val in data['doc_topic_dists'][-1]:
    ret_val += (str((val*100).round(2)) +'% *'+ str(topic_to_topwords[t][0])+'\n')
    t+=1



  return ret_val 



query = ""
for i in range(500):
  query += ' nucleic plaque carried protein kit assay extracted inhibits rna quantified '
query += ' ------------------------------ ' 
query += ' nucleic plaque carried protein kit assay extracted inhibits rna quantified ' 

comp = getTopicComposition(query)
for line in comp.split('\n'):
  print(line)

	 factors
	 weights
title:  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits rna quantified  nucleic plaque carried protein kit assay extracted inhibits 

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import Flask, jsonify
from flask import abort
from flask import make_response
from flask import request

app = Flask(__name__)
run_with_ngrok(app)
tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]


@app.route("/")
def home():
    return "<h1>Infer Topic Compositions on Google Colab!</h1>"

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    print(task['title'])
    composition = getTopicComposition(task['title'])
    print(composition) 
    return jsonify({'topic composition': str(composition)}), 201

app.run()
